In [ ]:
import re

# --- Helper functions ---

def is_variable(x):
    """Check if x is a variable (starts lowercase letter)."""
    return re.fullmatch(r'[a-z]\w*', x) is not None

def parse(expr):
    """
    Parse a logical expression like Knows(John, Mother(y))
    into a tuple structure, e.g. ('Knows', ['John', ('Mother', ['y'])])
    """
    expr = expr.strip()
    if '(' not in expr:
        return expr
    pred, args = expr.split('(', 1)
    args = args[:-1]  # remove closing parenthesis
    parts, depth, token = [], 0, ''
    for ch in args:
        if ch == ',' and depth == 0:
            parts.append(parse(token.strip()))
            token = ''
        else:
            if ch == '(':
                depth += 1
            elif ch == ')':
                depth -= 1
            token += ch
    if token:
        parts.append(parse(token.strip()))
    return (pred.strip(), parts)

def occurs_check(var, term, subs):
    """Return True if variable occurs in term (prevents x = Mother(x))"""
    term = substitute(term, subs)
    if var == term:
        return True
    if isinstance(term, tuple):
        return any(occurs_check(var, t, subs) for t in term[1])
    return False

def substitute(term, subs):
    """Apply current substitutions to a term."""
    if isinstance(term, str):
        return substitute(subs[term], subs) if term in subs else term
    return (term[0], [substitute(a, subs) for a in term[1]])

def pretty(term):
    """Convert a parsed term back to readable string."""
    if isinstance(term, str):
        return term
    return f"{term[0]}({', '.join(pretty(a) for a in term[1])})"


# --- Main Algorithm: Unify(1, 2) ---
def Unify(x, y, subs=None):
    """Implements your given Unify algorithm recursively."""
    if subs is None:
        subs = {}

    # Step 1: Base cases
    x = substitute(x, subs)
    y = substitute(y, subs)

    if x == y:
        return subs

    if isinstance(x, str) and is_variable(x):
        if occurs_check(x, y, subs):
            raise ValueError(f"Occurs check failed: {x} in {pretty(y)}")
        subs[x] = y
        return subs

    elif isinstance(y, str) and is_variable(y):
        if occurs_check(y, x, subs):
            raise ValueError(f"Occurs check failed: {y} in {pretty(x)}")
        subs[y] = x
        return subs

    elif isinstance(x, tuple) and isinstance(y, tuple):
        # Step 2: Check predicate symbol
        if x[0] != y[0]:
            raise ValueError(f"Predicate mismatch: {x[0]} vs {y[0]}")

        # Step 3: Check number of arguments
        if len(x[1]) != len(y[1]):
            raise ValueError("Different number of arguments.")

        # Step 4: Initialize substitution set
        # Step 5: For each argument, unify recursively
        for a1, a2 in zip(x[1], y[1]):
            subs = Unify(a1, a2, subs)
        return subs

    # Step 1.d: If none of the above, failure
    raise ValueError(f"Cannot unify {pretty(x)} with {pretty(y)}")


# --- Example run ---
if __name__ == "__main__":
    expr1 = "p(B,x,f(g(z)))"
    expr2 = "p(z,f(y),f(y))"

    X, Y = parse(expr1), parse(expr2)
    print("Expression 1:", expr1)
    print("Expression 2:", expr2)

    try:
        result = Unify(X, Y)
        print("\nUnification Successful")
        print("Substitution Set (MGU):", {k: pretty(v) for k, v in result.items()})
        unified = substitute(X, result)
        print("Unified Expression:", pretty(unified))
    except ValueError as e:
        print("\nUnification Failed:", e)


Expression 1: p(B,x,f(g(z)))
Expression 2: p(z,f(y),f(y))

Unification Successful
Substitution Set (MGU): {'z': 'B', 'x': 'f(y)', 'y': 'g(B)'}
Unified Expression: p(B, f(g(B)), f(g(B)))


In [8]:
import re

# --- Helper functions ---

def is_variable(x):
    """Check if x is a variable (starts lowercase letter)."""
    return re.fullmatch(r'[a-z]\w*', x) is not None

def parse(expr):
    """
    Parse a logical expression like Knows(John, Mother(y))
    into a tuple structure, e.g. ('Knows', ['John', ('Mother', ['y'])])
    """
    expr = expr.strip()
    if '(' not in expr:
        return expr
    pred, args = expr.split('(', 1)
    args = args[:-1]  # remove closing parenthesis
    parts, depth, token = [], 0, ''
    for ch in args:
        if ch == ',' and depth == 0:
            parts.append(parse(token.strip()))
            token = ''
        else:
            if ch == '(':
                depth += 1
            elif ch == ')':
                depth -= 1
            token += ch
    if token:
        parts.append(parse(token.strip()))
    return (pred.strip(), parts)

def occurs_check(var, term, subs):
    """Return True if variable occurs in term (prevents x = Mother(x))"""
    term = substitute(term, subs)
    if var == term:
        return True
    if isinstance(term, tuple):
        return any(occurs_check(var, t, subs) for t in term[1])
    return False

def substitute(term, subs):
    """Apply current substitutions to a term."""
    if isinstance(term, str):
        return substitute(subs[term], subs) if term in subs else term
    return (term[0], [substitute(a, subs) for a in term[1]])

def pretty(term):
    """Convert a parsed term back to readable string."""
    if isinstance(term, str):
        return term
    return f"{term[0]}({', '.join(pretty(a) for a in term[1])})"


# --- Main Algorithm: Unify(1, 2) ---
def Unify(x, y, subs=None):
    """Implements your given Unify algorithm recursively."""
    if subs is None:
        subs = {}

    # Step 1: Base cases
    x = substitute(x, subs)
    y = substitute(y, subs)

    if x == y:
        return subs

    if isinstance(x, str) and is_variable(x):
        if occurs_check(x, y, subs):
            raise ValueError(f"Occurs check failed: {x} in {pretty(y)}")
        subs[x] = y
        return subs

    elif isinstance(y, str) and is_variable(y):
        if occurs_check(y, x, subs):
            raise ValueError(f"Occurs check failed: {y} in {pretty(x)}")
        subs[y] = x
        return subs

    elif isinstance(x, tuple) and isinstance(y, tuple):
        # Step 2: Check predicate symbol
        if x[0] != y[0]:
            raise ValueError(f"Predicate mismatch: {x[0]} vs {y[0]}")

        # Step 3: Check number of arguments
        if len(x[1]) != len(y[1]):
            raise ValueError("Different number of arguments.")

        # Step 4: Initialize substitution set
        # Step 5: For each argument, unify recursively
        for a1, a2 in zip(x[1], y[1]):
            subs = Unify(a1, a2, subs)
        return subs

    # Step 1.d: If none of the above, failure
    raise ValueError(f"Cannot unify {pretty(x)} with {pretty(y)}")


# --- Example run ---
if __name__ == "__main__":
    expr1 = "p(Ji,x)"
    expr2 = "p(x,Jo)"

    X, Y = parse(expr1), parse(expr2)
    print("Expression 1:", expr1)
    print("Expression 2:", expr2)

    try:
        result = Unify(X, Y)
        print("\nUnification Successful")
        print("Substitution Set (MGU):", {k: pretty(v) for k, v in result.items()})
        unified = substitute(X, result)
        print("Unified Expression:", pretty(unified))
    except ValueError as e:
        print("\nUnification Failed:", e)


Expression 1: p(Ji,x)
Expression 2: p(x,Jo)

Unification Failed: Cannot unify Ji with Jo
